<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Academics/AI%20Project/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as ureq
ureq.urlretrieve('https://raw.githubusercontent.com/TXH2020/MainRepo/main/Academics/AI%20Project/intents.json','intents.json')

('intents.json', <http.client.HTTPMessage at 0x7fca300080d0>)

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
# things we need for Tensorflow
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype='object')

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
# reset underlying graph data
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.layers.Dense(8,input_shape=(len(train_x[0]),),kernel_initializer='normal'),
    keras.layers.Dense(8,kernel_initializer='normal'),
    keras.layers.Dense(len(train_y[0]),activation='softmax',kernel_initializer='normal')
])
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y), epochs=1000, batch_size=8, verbose=0)

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
['Accounts', 'Admin', 'Chemistry', 'Electrical', 'Electronics', 'FAQ', 'Hostel', 'MCA', 'Mathematics', 'Mechanical', 'Physics', 'Rajyotsava', 'apex', 'canteen', 'courses', 'cse', 'exam', 'goodbye', 'greeting', 'holiday', 'library', 'opentoday', 'parking', 'payments', 'principal', 'seats', 'seminar', 'sports', 'teaching', 'thanks', 'tomorrow']


In [9]:
print(model.predict(np.array([p])))

1/1 [==============================] - 0s 76ms/step
[[1.6809016e-02 1.0538009e-08 1.4300817e-03 6.4795779e-04 6.4531683e-05
  1.4821038e-11 1.9213090e-04 6.5644490e-09 1.7660875e-08 2.1676034e-07
  1.5799523e-08 7.7980351e-09 1.6683585e-11 6.1910277e-10 1.4766138e-10
  4.6447923e-08 2.1983096e-11 4.6257474e-03 1.1113927e-04 7.0850163e-08
  1.3133170e-07 9.7225249e-01 2.7629435e-06 6.7844676e-06 5.6213536e-09
  2.6495347e-03 9.7523437e-14 7.3422976e-11 3.1362069e-06 2.3842341e-07
  1.2039733e-03]]


In [11]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('m.tflite', 'wb') as f:
  f.write(tflite_model)

In [12]:
import pickle
with open('i.pkl','wb') as f:
  pickle.dump(intents,f)
with open('c.pkl','wb') as f:
  pickle.dump(classes,f)
with open('w.pkl','wb') as f:
  pickle.dump(words,f)